<a href="https://colab.research.google.com/github/Farhad-Davaripour/Navigating_GenAI/blob/main/function_calling/function_calling_demo.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Invoking an LLM

In [3]:
%pip install openai scikit-learn
%pip install openai openai

  Using cached openai-1.30.1-py3-none-any.whl (320 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [17]:
import os
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity

OPENAI_API_KEY=""

# Set the environment variable
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI(
  api_key=OPENAI_API_KEY
)

In [5]:
# Generate response using the language model
def create_prompt(prompt):
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that provides answer to the user's query."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"{prompt}"
                }
            ]
        },
    ]

In [6]:
def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",  # Replace with the appropriate model name
        messages=create_prompt(prompt),
        max_tokens=150
    )
    return response

def print_response(prompt):
    response = generate_response(prompt)
    for key in response.choices[0]:
        print(key)
    print("\nResponse: ", response.choices[0].message.content)

In [7]:
response = print_response("What is the capital of Canada")

('finish_reason', 'stop')
('index', 0)
('logprobs', None)
('message', ChatCompletionMessage(content='The capital of Canada is Ottawa.', role='assistant', function_call=None, tool_calls=None))

Response:  The capital of Canada is Ottawa.


In [8]:
x_values = [1, 2, 3, 4, 5]
y_values = [10, 20, 25, 30, 35]

response = print_response(f'generate a scatter plot with x values {x_values} and y values {y_values}')

('finish_reason', 'length')
('index', 0)
('logprobs', None)
('message', ChatCompletionMessage(content="Generating a scatter plot involves creating a plot where each point represents a pair of `x` and `y` values on a Cartesian coordinate system. The following Python code uses the popular plotting library Matplotlib to achieve this:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Define the data\nx_values = [1, 2, 3, 4, 5]\ny_values = [10, 20, 25, 30, 35]\n\n# Create the scatter plot\nplt.scatter(x_values, y_values)\n\n# Add titles and labels\nplt.title('Scatter Plot of X and Y Values')\nplt.xlabel('X Values')\nplt.ylabel('Y Values')\n\n# Display the plot\nplt.show()\n```\n\nTo use this", role='assistant', function_call=None, tool_calls=None))

Response:  Generating a scatter plot involves creating a plot where each point represents a pair of `x` and `y` values on a Cartesian coordinate system. The following Python code uses the popular plotting library Matplotlib to achieve this:

``

# Function Calling

In [11]:
%pip install matplotlib langchain langchain_core langchain_openai

In [12]:
import matplotlib.pyplot as plt
from langchain.agents import tool

@tool
def scatter_plot(x: list, y: list):
    """
    Plot a scatter plot using two lists: x and y
    :param x: List of x values
    :param y: List of y values
    """


    if len(x) != len(y):
        raise ValueError("The length of x and y must be the same")

    plt.scatter(x, y)
    plt.xlabel('X values')
    plt.ylabel('Y values')
    plt.title('Scatter Plot of X vs Y')
    plt.grid(True)
    plt.show()


In [13]:
tools = [scatter_plot]

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant that provides answer to the user's query.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [18]:
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

llm = ChatOpenAI(temperature=0.1, model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, handle_parsing_errors=True
)

In [19]:
def agent_executor_fn(prompt: str) -> str:
    return list(agent_executor.stream({"input": prompt}))

In [ ]:
x_values = [1, 2, 3, 4, 5]
y_values = [10, 20, 25, 30, 35]

for key in response:
    print(key)

response = agent_executor_fn(f'plot a scatter plot with x values {x_values} and y values {y_values}')